<a href="https://colab.research.google.com/github/Zeeshan0123/LLM_Apps/blob/main/CogvideoX_Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade diffusers accelerate transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [2]:
!pip install torch torchvision --force-reinstall --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.1/799.1 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 100.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 104.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import torch
import gc
from diffusers import CogVideoXPipeline, CogVideoXDDIMScheduler, CogVideoXDPMScheduler
from diffusers.utils import export_to_video
from transformers import T5EncoderModel, T5Tokenizer

In [3]:
from huggingface_hub import login
login()

In [2]:
def reset_memory():
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_accumulated_memory_stats()
    torch.cuda.reset_peak_memory_stats()

In [3]:
def print_memory():
    memory = round(torch.cuda.memory_allocated() / 1024**3, 2)
    max_memory = round(torch.cuda.max_memory_allocated() / 1024**3, 2)
    max_reserved = round(torch.cuda.max_memory_reserved() / 1024**3, 2)
    print(f"{memory=} GB")
    print(f"{max_memory=} GB")
    print(f"{max_reserved=} GB")

In [4]:
prompt = (
    "Generate an video of a serene forest with tall trees and a flowing river, with a waterfall in the background, an Islamic mosque nestled among the foliage, and a bird perched on a branch, during a huge storm with strong winds and heavy rain, in a style that combines elements of Vincent van Gogh and minimalistic. Add blue tones, lit with natural light."
)

In [5]:
pipe = CogVideoXPipeline.from_pretrained("THUDM/CogVideoX-2b", torch_dtype=torch.float16)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

The config attributes {'mid_block_add_attention': True} were passed to AutoencoderKLCogVideoX, but are not expected and will be ignored. Please verify your config.json configuration file.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
pipe

CogVideoXPipeline {
  "_class_name": "CogVideoXPipeline",
  "_diffusers_version": "0.30.0",
  "_name_or_path": "THUDM/CogVideoX-2b",
  "scheduler": [
    "diffusers",
    "CogVideoXDDIMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "T5EncoderModel"
  ],
  "tokenizer": [
    "transformers",
    "T5Tokenizer"
  ],
  "transformer": [
    "diffusers",
    "CogVideoXTransformer3DModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKLCogVideoX"
  ]
}

In [8]:
# from diffusers import PixArtTransformer2DModel
# pipe.transformer = PixArtTransformer2DModel()

In [6]:
# pipe.scheduler = CogVideoXDDIMScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")
pipe.scheduler = CogVideoXDPMScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")

In [7]:
text_tokenizer_id = "google/t5-v1_1-small"
pipe.tokenizer = T5Tokenizer.from_pretrained(text_tokenizer_id)
# pipe.text_encoder = T5EncoderModel.from_pretrained(text_encoder_id)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
# from diffusers import AutoencoderKLCogVideoX

# vae = AutoencoderKLCogVideoX.from_pretrained("THUDM/CogVideoX-2b", subfolder="vae", torch_dtype=torch.float16).to("cuda")
from diffusers import AutoencoderKL

pipe.vae = AutoencoderKL()

In [12]:
# from transformers import T5EncoderModel

# pipe.text_encoder = T5EncoderModel.from_pretrained("google-t5/t5-small")


In [9]:
pipe.enable_model_cpu_offload()

In [10]:
pipe.vae.enable_tiling()

In [10]:
# pipe.vae.enable_slicing()

In [9]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [10]:
torch._inductor.config.conv_1x1_as_mm = True
torch._inductor.config.coordinate_descent_tuning = True
torch._inductor.config.epilogue_fusion = False
torch._inductor.config.coordinate_descent_check_all_directions = True

In [11]:
pipe.transformer.to(memory_format=torch.channels_last)

CogVideoXTransformer3DModel(
  (patch_embed): CogVideoXPatchEmbed(
    (proj): Conv2d(16, 1920, kernel_size=(2, 2), stride=(2, 2))
    (text_proj): Linear(in_features=4096, out_features=1920, bias=True)
  )
  (embedding_dropout): Dropout(p=0.0, inplace=False)
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=1920, out_features=512, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=512, out_features=512, bias=True)
  )
  (transformer_blocks): ModuleList(
    (0-29): 30 x CogVideoXBlock(
      (norm1): CogVideoXLayerNormZero(
        (silu): SiLU()
        (linear): Linear(in_features=512, out_features=11520, bias=True)
        (norm): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
      )
      (attn1): Attention(
        (norm_q): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
        (norm_k): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
        (to_q): Linear(in_features=1920, out_features=1920, bi

In [12]:
pipe.transformer = torch.compile(pipe.transformer, mode="max-autotune", fullgraph=True)

In [ ]:
# pipe = pipe.to(dtype=torch.bfloat16)

In [11]:
print_memory()

memory=0.0 GB
max_memory=0.0 GB
max_reserved=0.0 GB


In [12]:
reset_memory()

In [11]:
# prompt: full code to empty the system ram  not gpu system ram

import gc

def empty_system_ram():
  gc.collect()

empty_system_ram()


In [14]:
pipe

CogVideoXPipeline {
  "_class_name": "CogVideoXPipeline",
  "_diffusers_version": "0.30.0",
  "_name_or_path": "THUDM/CogVideoX-2b",
  "scheduler": [
    "diffusers",
    "CogVideoXDPMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "T5EncoderModel"
  ],
  "tokenizer": [
    "transformers",
    "T5Tokenizer"
  ],
  "transformer": [
    "diffusers",
    "CogVideoXTransformer3DModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
generator = torch.Generator().manual_seed(32)

video = pipe(prompt=prompt,
             guidance_scale=2,
             num_frames=32,
             num_inference_steps=3,
             generator=generator)

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Print the shape of the latents tensor
print(video.latents.shape)


In [ ]:
video = video.frames[0]

In [ ]:
export_to_video(video, "output.mp4", fps=8)

In [20]:
# prompt: full code to empty the system ram  not gpu system ram

import gc

def empty_system_ram():
  gc.collect()

empty_system_ram()
